In [ ]:
# 使用集成学习，建立两个模型
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D
# 一个加入比较好的
class Cifar_TWO_CONV2_norm(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_norm, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(32)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(64)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=4096, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
        

class Cifar_TWO_CONV2_lenet(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_lenet, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(32)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(64)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc1 = Linear(in_features=4096, out_features=2048)
        self.fc2 = Linear(in_features=2048, out_features=1024)
        self.fc3 = Linear(in_features=1024, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc1(x)
        x = F.dropout(x,p =0.1,mode=mode )
        x = self.fc2(x)
        x = F.dropout(x, p=0.1, mode=mode)
        x = self.fc3(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

In [ ]:
# 定义训练过程
from collections import Counter
import numpy as np
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D

logwriter = LogWriter(logdir="./run/cifar_log/two_conv_jicheng")
def train_Cifar_TWO2_norm_end():
    # 初始化网络
    net1 = Cifar_TWO_CONV2_lenet()
    net1.train()
    net2 = Cifar_TWO_CONV2_norm()
    net_test = Cifar_ONE_CONV()
    # 两个个模型放一起
    mlps = [net1, net2]
    opt1 = paddle.optimizer.Adam( learning_rate=0.01, parameters=net1.parameters())
    opt2 = paddle.optimizer.Adam( learning_rate=0.01, parameters=net2.parameters())
    opt = [opt1, opt2]
    # 定义优化器
    EPOCH_NUM = 15

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)
            for i, mlp in enumerate(mlps):
                mlp.train()
                # 前向
                net_test.train()
                predict,acc = net_test(images,labels)
                net1.train()
                predict,acc = net1(images,labels,mode='upscale_in_train')
                predict, acc = mlp(images, labels)

                # 损失
                loss = F.cross_entropy(predict, labels)
                avg_loss = paddle.mean(loss)

                # 反向传播
                avg_loss.backward()
                opt[i].step()
                opt[i].clear_grad()
        pre=[]
        vote_correct = 0
        mlps_correct = [0 for i in range(len(mlps))]
        for batch_id, data in enumerate(test_loader()):
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)
            for i, mlp in enumerate(mlps):
                mlp.eval()
                
                # 预测
                predicts, acc = mlp(images, labels,mode='downscale_in_infer')
                _, prediction = paddle.max(predicts,1)
                pre_num = prediction.numpy()
                mlps_correct[i] +=(pre_num==labels.numpy()).sum
                pre.append(pre_num)
            arr = np.array(pre)
            pre.clear()
            result = [Counter(arr[:, i]).most_common(1)[0][0] for i in range(128)]
            vote_correct += (result == labels.numpy()).sum()

        print("epoch:" + str(epoch_id) + "集成模型的正确率" + str(vote_correct / len(test_loader)))
 
        for idx, coreect in enumerate(mlps_correct):
            print("模型" + str(idx) + "的正确率为：" + str(coreect / len(test_loader)))